# Libraries:
https://pymotw.com/2/socket/multicast.html

In [1]:
import socket
import netifaces
import struct
import json

# Functions:

In [5]:
def getComputerInfo():
    macAddresses = list(
        filter(
            None,
            [netifaces.ifaddresses(interface)[netifaces.AF_LINK][0]['addr']
             for interface
             in netifaces.interfaces()
            ]
        )
    )
    computerName = socket.gethostname()
    ipAddress = socket.gethostbyname(computerName)
    
    return {
        computerName : {
            "Mac Addresses" : macAddresses,
            "IP Address" : ipAddress
        }
    }

# Mutlicast:

## Sender:

In [7]:
multicast_group = ("224.3.29.71",10000)

# Create datagram socket
sock = socket.socket(socket.AF_INET, 
                     socket.SOCK_DGRAM)

# Set timeout
sock.settimeout(0.2)

# Set time-to-live
ttl = struct.pack('b', 1)
sock.setsockopt(socket.IPPROTO_IP, 
                socket.IP_MULTICAST_TTL, 
                ttl)

data = bytes(str(getComputerInfo()), "utf-8")

try:
    
    sent = sock.sendto(data, multicast_group)
    
    # Look for responses from all recipients
    while True:
        try:
            data, address = sock.recvfrom(1024)
            foreignComputerData = json.loads(data.decode("utf-8").replace("'",'"'))
            print(foreignComputerData)
            
        except socket.timeout:
            break
finally:
    sock.close()

# Receiver:

In [ ]:
multicast_group = "224.3.29.71"
server_address = ("", 10000)

# Create datagram socket
sock = socket.socket(socket.AF_INET, 
                     socket.SOCK_DGRAM)

# Bind to server address
sock.bind(server_address)

# Tell the OS to add socket to multicasr groupon all interfaces
group = socket.inet_aton(multicast_group)
mreq = struct.pack("4sL", group, socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, 
                socket.IP_ADD_MEMBERSHIP, 
                mreq)

# Receive/respond loop
while True:
    data, address = sock.recvfrom(1024)
    foreignComputerData = json.loads(data.decode("utf-8").replace("'",'"'))
    print(foreignComputerData)
    data = bytes(str(getComputerInfo()),"utf-8")
    sock.sendto(data, address)

# Server:

In [4]:
def main(IP):
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.bind((IP,2142))
    s.listen(5)
    while True:
        clientsocket, address = s.accept()
        msg = input(">>> ")
        s.sendto(bytes(msg,"UTF-8"), IP)
        break
main("192.168.1.155")

OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

# Client:

In [5]:
def main(IP):
    HEADERSIZE = 10
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.connect((IP,2142))
    
    msg = s.recv(16)
    msg = msg.decode("utf-8")
    print (msg)
main("192.168.1.155")